In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') #, force_remount=True

Mounted at /content/gdrive


In [ ]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks/miramedix')

In [ ]:
pip install pyserini

In [ ]:
pip install faiss

In [ ]:
pip install faiss-cpu

In [5]:
import os
from pyserini.dsearch import SimpleDenseSearcher
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
import json

with open('descriptions.json', 'r') as f:
  texts = json.load(f)

batch 20 (если много документов)

In [ ]:
! python -m pyserini.encode input --corpus sample_collection_jsonl_2/doc_75.jsonl output --embeddings indexes_2/dindex-sample-dpr-multi75 --to-faiss encoder --encoder facebook/dpr-ctx_encoder-multiset-base --device cpu

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-multiset-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenize

In [ ]:
num = '75'

In [ ]:
query = texts[num]

In [ ]:
input_path = 'indexes_2/dindex-sample-dpr-multi'+num
print(input_path)

searcher = SimpleDenseSearcher(input_path, 'facebook/dpr-question_encoder-multiset-base')

indexes/dindex-sample-dpr-multi75
pyserini.dsearch.SimpleDenseSearcher class has been deprecated, please use FaissSearcher from pyserini.search.faiss instead


Some weights of the model checkpoint at facebook/dpr-question_encoder-multiset-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
hits = searcher.search(query)
hits_list = []

for i in range(0, len(hits)):    
    hits_list.append([hits[i].docid, str(hits[i].score)])

In [ ]:
file_name = f'reranking_2/rerank_{num}.json'
with open(file_name, 'w') as f:
  json.dump(hits_list, f)